#### Câu 1. Viết câu lệnh SQLđể tính sự tương quan giữa A và B theo công thức

In [18]:
import pandas as pd
import numpy as np

# Giả sử ta có DataFrame df với hai cột 'A' và 'B'
#Dữ liệu mẫu
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

#code theo công thức

def calculate_correlation(df):
    a = df['A']
    b = df['B']
    n = len(df)
    
    sum_ab = np.sum(a * b)
    sum_a = np.sum(a)
    sum_b = np.sum(b)
    sum_a2 = np.sum(a ** 2)
    sum_b2 = np.sum(b ** 2)

    numerator = n * sum_ab - sum_a * sum_b
    denominator = np.sqrt(n * sum_a2 - sum_a**2) * np.sqrt(n * sum_b2 - sum_b**2)

    r = numerator / denominator
    return r

# Gọi hàm
correlation = calculate_correlation(df)
print("Hệ số tương quan Pearson:", correlation)


Hệ số tương quan Pearson: 1.0000000000000002


In [19]:
# Dùng hàm có sẵn của pandas
correlation = df['A'].corr(df['B'])
print("Hệ số tương quan Pearson:", correlation)


Hệ số tương quan Pearson: 1.0


#### Câu 2

##### Chuyển đổi dữ liệu về dạng quan hệ (long format)

In [20]:
import pandas as pd

# Tạo bảng dữ liệu
data = {
    'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4'],
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6],
    'C': [7, 7, 8, 5]
}

df = pd.DataFrame(data)

# Chuyển sang dạng quan hệ
df_long = pd.melt(df, id_vars='Day', var_name='Model', value_name='Score')      #dùng pandas.melt() để chuyển từ bảng ngang thành bảng dọc
print(df_long)


      Day Model  Score
0   Day 1     A    8.0
1   Day 2     A    7.5
2   Day 3     A    6.0
3   Day 4     A    7.0
4   Day 1     B    9.0
5   Day 2     B    8.5
6   Day 3     B    7.0
7   Day 4     B    6.0
8   Day 1     C    7.0
9   Day 2     C    7.0
10  Day 3     C    8.0
11  Day 4     C    5.0


##### Phân nhóm điểm (rời rạc hóa)

In [21]:
# Chia điểm thành nhóm: Low (0-6], Medium (6-8], High (8-10]
df_long['ScoreGroup'] = pd.cut(df_long['Score'], bins=[0, 6, 8, 10], labels=['Low', 'Medium', 'High'])

##### Tạo bảng tần số và kiểm định Chi bình phương ($X^2$) 

In [22]:
import scipy.stats as stats

# Bảng tần số giữa Model và ScoreGroup
contingency = pd.crosstab(df_long['Model'], df_long['ScoreGroup'])

# Kiểm định Chi bình phương
chi2, p, dof, expected = stats.chi2_contingency(contingency)

print("Bảng tần số:\n", contingency)
print("Chi-square:", chi2)
print("p-value:", p)


Bảng tần số:
 ScoreGroup  Low  Medium  High
Model                        
A             1       3     0
B             1       1     2
C             1       3     0
Chi-square: 5.142857142857143
p-value: 0.2729510249670288


**Nhận xét:** Vì p-value = 0.273 > 0.05, nên có thể thấy không có sự khác biệt đáng kể giữa các mẫu xe. Kết quả thử nghiệm không phụ thuộc vào mẫu xe

#### Phân tích xem kết quả có phụ thuộc vào ngày không

In [23]:
# Bảng tần số: hàng là Day, cột là nhóm điểm
contingency_day = pd.crosstab(df_long['Day'], df_long['ScoreGroup'])

# Kiểm định Chi bình phương theo ngày
chi2_day, p_day, dof_day, expected_day = stats.chi2_contingency(contingency_day)

print("Bảng tần số theo ngày:\n", contingency_day)
print("Chi-square (Day):", chi2_day)
print("p-value (Day):", p_day)


Bảng tần số theo ngày:
 ScoreGroup  Low  Medium  High
Day                          
Day 1         0       2     1
Day 2         0       2     1
Day 3         1       2     0
Day 4         2       1     0
Chi-square (Day): 6.095238095238095
p-value (Day): 0.41260652765580624


**Nhận xét:** Vì p-value = 0.413 > 0.05, nên ta thấy không có sự khác biệt đáng kể về điểm số giữa các ngày thử nghiệm.
Kết quả chấm điểm không phụ thuộc vào thời gian/ngày kiểm tra.

#### Câu 3

##### Chuyển đổi từ số nguyên thành định dạng thời gian

In [24]:
import pandas as pd

# Giả sử đây là dữ liệu ban đầu
data = {'departure_time': [830, 1445, 920, 1720, 2355]}
df = pd.DataFrame(data)

print(df)


   departure_time
0             830
1            1445
2             920
3            1720
4            2355


##### Chuyển đổi thời gian

In [25]:
# Hàm chuyển đổi số nguyên thành thời gian dạng 'HH:MM'
def convert_to_time(departure_time):
    # Lấy giờ và phút từ giá trị số nguyên
    hour = departure_time // 100
    minute = departure_time % 100
    return f"{hour:02}:{minute:02}"

# Áp dụng hàm chuyển đổi cho cột departure_time
df['departure_time'] = df['departure_time'].apply(convert_to_time)

# Hiển thị kết quả
print(df)


  departure_time
0          08:30
1          14:45
2          09:20
3          17:20
4          23:55


##### Chuyển sang kiểu dữ liệu datetime

In [26]:
import pandas as pd
from datetime import datetime

# Chuyển đổi departure_time thành datetime object với ngày hiện tại
df['departure_time'] = pd.to_datetime(df['departure_time'], format='%H:%M').apply(lambda x: x.replace(year=datetime.now().year, month=datetime.now().month, day=datetime.now().day))

# Kiểm tra kết quả
print(df)


       departure_time
0 2025-04-07 08:30:00
1 2025-04-07 14:45:00
2 2025-04-07 09:20:00
3 2025-04-07 17:20:00
4 2025-04-07 23:55:00


##### Câu 4

In [27]:
import numpy as np
import pandas as pd

# TÍnh MAD
median = df_long['Score'].median()
mad = (df_long['Score'] - median).abs().median()

# TÌm các ngoại lệ (giá trị lớn hơn 1.5 lần MAD)

threshold = 1.5 * mad
outliners = df_long[(df_long['Score'] < median - threshold) | (df_long['Score'] > median + threshold)]

print("Các giá trị ngoại lệ:")
print(outliners)

Các giá trị ngoại lệ:
      Day Model  Score ScoreGroup
4   Day 1     B    9.0       High
11  Day 4     C    5.0        Low


#### Nhận xét

Trong dữ liệu, các giá trị ngoại lệ được tìm thấy là:
- **Model B, Day 1, Score = 9.0 (High)**: Điểm này cao hơn ngưỡng trên của khoảng cho phép.
- **Model C, Day 4, Score = 5.0 (Low)**: Điểm này thấp hơn ngưỡng dưới của khoảng cho phép.



#### Câu 5

In [ ]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

# Tạo bảng Patient với dữ liệu mẫu
data = {
    'last_name': ['Nguyen', 'Nguyen', 'Tran', 'Le'],  # Họ của bệnh nhân
    'weight': [70, 70, 80, 60],  # Cân nặng của bệnh nhân
    'height': [175, 175, 180, 165]  # Chiều cao của bệnh nhân
}
df_patient = pd.DataFrame(data)

# Hàm tính khoảng cách Boolean
def boolean_distance(u, v):
    """
    Hàm tính khoảng cách Boolean giữa hai vector u và v.
    Khoảng cách Boolean trả về True (1) nếu hai vector không khớp hoàn toàn
    dựa trên 'last_name' và 'weight', ngược lại trả về False (0).
    """
    last_name_match = u[0] == v[0]  # So sánh họ của hai bệnh nhân
    weight_match = u[1] == v[1]  # So sánh cân nặng của hai bệnh nhân
    return not (last_name_match and weight_match)  # Trả về True nếu không khớp hoàn toàn

# Chuyển đổi cột 'last_name' thành giá trị số để dễ dàng tính toán
df_patient['last_name_encoded'] = df_patient['last_name'].astype('category').cat.codes

# Sử dụng cột mã hóa 'last_name_encoded' và cột 'weight' để tính khoảng cách
distances = pdist(df_patient[['last_name_encoded', 'weight']].values, metric=boolean_distance)

# Tính ma trận khoảng cách
# Chuyển đổi khoảng cách thành ma trận vuông để dễ dàng quan sát
distance_matrix = squareform(distances)

# Hiển thị bảng dữ liệu và ma trận khoảng cách
print("Bảng dữ liệu mẫu:")
print(df_patient)
print("\nMa trận khoảng cách Boolean:")
print(distance_matrix)

Bảng dữ liệu mẫu:
  last_name  weight  height  last_name_encoded
0    Nguyen      70     175                  1
1    Nguyen      70     175                  1
2      Tran      80     180                  2
3        Le      60     165                  0

Ma trận khoảng cách Boolean:
[[0. 0. 1. 1.]
 [0. 0. 1. 1.]
 [1. 1. 0. 1.]
 [1. 1. 1. 0.]]


#### Nhận xét 


Kết quả trên cho thấy:
- Bệnh nhân 0 và 1 có cùng họ "Nguyen" và cân nặng 70, nên khoảng cách giữa họ là `0`.
- Bệnh nhân 0 và 2 khác nhau về cả họ ("Nguyen" vs "Tran") và cân nặng (70 vs 80), nên khoảng cách giữa họ là `1`.


